<a href="https://colab.research.google.com/github/dfridland/PyTorch/blob/HW9/DF_HW_PyTorch9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers stop_words pymorphy2 tqdm nltk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 35.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 106.4 MB/s eta 0:00:00
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=b8e182b27ca41d6d097f8713efd80880cdd0b0dea2c62d20895c5df025fa6697
  Stored in directory: /root/.cache/pip/wheels/da/d8/66/395317506a23a9d1d7de433ad6a7d9e6e16aab48cf028a0f60
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=74f405af38cbb073e8ac2675cb4e53b9ec6e31b40ec927441c1ae3d76a3084

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

import torch
import torch.nn as nn

from torch.optim import Adam
from tqdm import tqdm

from transformers import AutoModelForSequenceClassification
from transformers import BertForSequenceClassification
from transformers import BertTokenizerFast

from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from string import punctuation
import nltk

In [3]:
nltk.download("punkt")
stop_words = set(get_stop_words("ru"))
punctuations = set(punctuation)
lemmatizer = MorphAnalyzer()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


'cuda'

In [4]:
tok = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
b_model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)


In [5]:
class TwitterDataset(torch.utils.data.Dataset):

    def __init__(self, txts, labels, tokenizer):
        self._labels = labels

        # self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.tokenizer = tokenizer
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]

    def __len__(self):
        return len(self._txts)

    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [6]:
class BertClassifier(nn.Module):

    def __init__(self, bert_model, dropout=0.5):
        super().__init__()
        # self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.Xrobert = bert_model
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(3, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):

        pooled_output = self.Xrobert(input_ids=x, attention_mask=mask, return_dict=False)[0]
        # _, pooled_output - набор эмбеддинигов слов, эмбеддинг предложения
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)

        return final_layer

In [7]:
def preprocess_text(txt, sw, lem, punct):

    txt = str(txt)
    txt = "".join(c for c in txt if c not in punct)
    txt = txt.lower()
    txt = [lem.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [8]:
!wget 'https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901' -O data.zip
!unzip data.zip

--2023-04-26 09:49:17--  https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901
Resolving drive.google.com (drive.google.com)... 172.253.122.101, 172.253.122.139, 172.253.122.113, ...
Connecting to drive.google.com (drive.google.com)|172.253.122.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/676g86hi5pn130hrs0sgpd6ksormtafm/1682502525000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=03d1fb30-eb15-42d9-8aa9-bfec674ddde9 [following]
--2023-04-26 09:49:18--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/676g86hi5pn130hrs0sgpd6ksormtafm/1682502525000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=03d1fb30-eb15-42d9-8aa9-bfec674ddde9
Resolving doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 142.251.167.132, 

In [9]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

df_train.shape, df_val.shape

((181467, 3), (22683, 3))

In [10]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

In [11]:
tqdm.pandas()

df_train['text'] = df_train['text'].progress_apply(lambda t: preprocess_text(t, stop_words, lemmatizer, punctuations))
df_val['text'] = df_val['text'].progress_apply(lambda t: preprocess_text(t, stop_words, lemmatizer, punctuations))

100%|██████████| 22683/22683 [00:25<00:00, 875.58it/s] 


In [12]:
%%time
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train, tok)
valid_dataset = TwitterDataset(df_val['text'], y_val, tok)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

CPU times: user 41.9 s, sys: 6.28 s, total: 48.2 s
Wall time: 46.2 s


In [13]:
for txt, lbl in train_loader:
    print(txt.keys())
    print(txt['input_ids'].shape)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([64, 1, 10])


In [14]:
model = BertClassifier(b_model).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.Xrobert.classifier.parameters(), lr=0.0016)

In [15]:
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.Xrobert.classifier.parameters()]))

BertClassifier(
  (Xrobert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=7

In [16]:
for epoch_num in range(2):
    total_acc_train = 0
    total_loss_train = 0
    model.train()

    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()

    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()

        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc

    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

100%|██████████| 2836/2836 [04:46<00:00,  9.91it/s]


Epochs: 1 | Train Loss:  0.011         | Train Accuracy:  0.559         | Val Loss:  0.011         | Val Accuracy:  0.582


100%|██████████| 2836/2836 [04:47<00:00,  9.88it/s]


Epochs: 2 | Train Loss:  0.011         | Train Accuracy:  0.559         | Val Loss:  0.011         | Val Accuracy:  0.582
